resultados(Nclusters, model_type, model, cl_method, tipo de distancia, overall same_c/n, overall diff_c/m, test same_c/n, test diff_c/m, handmade)

crear viualizaciones sobre esta tabla
puedo crear un sub-testdb de solo las preguntas que voy a tener en el conjunto de test

¿que se medira?

overall score
test score
handmade test score


separo en 2

mediciones(m_id, Nclusters, model_type, model, cl_method, tipo de distancia)

resultados(m_id, overall same_c/n, overall diff_c/m, test same_c/n, test diff_c/m, handmade)

10 20 30 40
tdidf, w2v, pretrained embbs
[Qkeys, Qclean, Q-ngrams, QA-keys, Qstp], [Qkeys, Qclean, QA-keys], [SBERT-Q, glove-Q]
kmeans
s2_distance (faiss default), cosine_distance



In [ ]:
import pandas as pd
import numpy as np
#import fasttext
from patterns import emoji_pattern
from gensim.models import Word2Vec, KeyedVectors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold
from sentence_transformers import SentenceTransformer, util


df = pd.read_csv("db/features.csv", index_col=0, converters={'Qtoks': pd.eval,'Qstp': pd.eval,'Qkeys': pd.eval}).dropna()

df

In [ ]:
def build_models(df, gpu=False):
    Models = {}

    tdidf = {}

    tdidf["Qkeys"] = tdidf_model(df, column="Q-kstr", min_df=15)
    tdidf["Qclean"] = tdidf_model(df, column="Qclean", min_df=15)
    tdidf["Q-ngrams"] = tdidf_model(df, column="Qclean", range=(2,3), min_df=15, ngram = True)
    tdidf["QA-keys"] = tdidf_model(df, column="QA-kstr", min_df=15)
    tdidf["Qstp"] = tdidf_model(df, column="Q-stpstr", min_df=1)

    Models["tdidf"] = tdidf

    w2v = {}

    w2v["Qclean"] = w2v_doc_embedding(list(df["Qclean"].astype("str")))
    w2v["Qkeys"] = w2v_doc_embedding(list(df["Qkeys"]))
    w2v["QA-keys"] = w2v_doc_embedding(list(df["QA-keys"]))

    Models["w2v"] = w2v

    if gpu:
        pretrained = {}

        glove60 = KeyedVectors.load_word2vec_format("glove.6B.300d.txt", binary=False, no_header=True)
        sbert_model = SentenceTransformer('nreimers/MiniLM-L6-H384-uncased') #
        
        pretrained["glove-Q"] = glove_doc_embedding(list(df["Qtoks"]), glove60)
        pretrained["SBERT-Q"] = sbert_model.encode(list(df["Q"].astype('str')), batch_size=70, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)

        Models["pretrain"] = pretrained

    return Models

def tdidf_model(df, column, range=(2,3), min_df=20, threshold=1e-3, ngram=False):
    
    docs = df[column].fillna("")

    if ngram:
        tdidf = TfidfVectorizer(min_df=min_df, ngram_range=range)
    else:
        tdidf = TfidfVectorizer(min_df=min_df)

    #Create, Normalize and Reduce 
    model = tdidf.fit_transform(docs)
    model = normalize(model, axis=1, norm="max")
    model = VarianceThreshold(threshold=threshold).fit_transform(model)

    return model.toarray()


def w2v_doc_embedding(docs_toks):

    docs_model = []
    
    words_model = Word2Vec(docs_toks, size=300, workers=8, seed=0)

    for tokens in docs_toks:
        zero_vector = np.zeros(words_model.vector_size)
        vectors = []
        for token in tokens:
            if token in words_model.wv:
                try:
                    vectors.append(words_model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            docs_model.append(avg_vec)
        else:
            docs_model.append(zero_vector)
    return np.array(docs_model)

def glove_doc_embedding(docs_toks, words_model):

        docs_model = []
        

        for tokens in docs_toks:
            zero_vector = np.zeros(words_model.vector_size)
            vectors = []
            for token in tokens:
                try:
                    vectors.append(words_model[token])
                except KeyError:
                    continue
            if vectors:
                vectors = np.asarray(vectors)
                avg_vec = vectors.mean(axis=0)
                docs_model.append(avg_vec)
            else:
                docs_model.append(zero_vector)
        return docs_model


In [ ]:
10 20 30 40
